## Environment Setup

In [ ]:
# Install dependencies
!pip install openai wikipedia

In [ ]:
import os
import json
import openai

# Retrieve credentials to access openai
with open('credentials.json') as c:
    data = json.load(c)
    api_key = data['OPENAI_API_KEY']
    
openai.api_key = api_key #os.environ.get("OPENAI_API_KEY")

if not openai.api_key:
    raise ValueError("No OpenAI API key found! Make sure OPENAI_API_KEY is set as a Codespaces secret.")

print("✅ OpenAI API key loaded successfully!")

## Define Simple Default Tools

In [ ]:
# --- Tools ---

import wikipedia

def calculator(expression: str) -> str:
    """Evaluates basic math expressions."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def wiki_search(query: str) -> str:
    """Fetches short summaries from Wikipedia."""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Error: {e}"

tools = {
    "calculator": calculator,
    "wiki": wiki_search,
}

print("✅ Tools ready: calculator(), wiki_search()")

## Define Agent Engine (ReAct Loop)

In [ ]:
# Define the agent's system prompt with an interchangeable tone/personality (you will see later on, why we do this)

def inject_system_prompt(agent_tone: str = None) -> str:
    # Assign the tone/personality if given, else fall back to default
    tone = agent_tone if agent_tone else "You are an intelligent agent that can reason step by step."

    # Dynamically gather tool descriptions to allow for new tools to be added later on
    tool_descriptions = "\n".join([f"- {name}(): {fn.__doc__ or 'no description'}" for name, fn in tools.items()])

    return f"""{tone}
    You have access to the following tools:
    {tool_descriptions}

    Follow this reasoning format strictly and repeat it, until you find the final answer:
    Thought: describe your reasoning
    Action: choose from the available tools (don't forget to pass the appropriate argument) or ANSWER()
    Observation: output/result of the previous action

    Rules:
    - Use at most ONE tool per step.
    - If you already know the answer, respond with:
    Action: ANSWER(<final answer here>)
    - Do NOT repeat the same tool call multiple times.
    - Stop once you have given the final answer.
    """ 
# TODO: Add line break

In [ ]:
# --- Agent Function ---

# TODO: Prevent double output of observation
def run_agent(query: str, max_steps: int = 3, agent_tone: str = None):
    print(f"User: {query}\n")

    context = inject_system_prompt(agent_tone=agent_tone)

    for step in range(1, max_steps + 1):
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": f"User question: {query}\nPrevious reasoning:\n{context}"}
            ]
        )

        text = response.choices[0].message.content.strip()
        print(text)

        lines = text.splitlines()
        action_line = next((l for l in lines if l.startswith("Action:")), None)
        if not action_line:
            print("\nNo action detected. Stopping.")
            break

        action = action_line.replace("Action:", "").strip()

        if "ANSWER" in action:
            final = action.split("ANSWER")[-1].strip(" :()")
            print(f"\nFinal Answer: {final}")
            return

        if "(" in action and action.endswith(")"):
            tool_name = action.split("(")[0]
            tool_input = action[len(tool_name) + 1:-1]

            if tool_name in tools:
                observation = tools[tool_name](tool_input)
            else:
                observation = f"Unknown tool: {tool_name}"
            context += f"\n{text}\nObservation: {observation}\n"
        else:
            print("\nCould not parse tool call. Stopping.")
            break

## Try It Out Yourself

In [ ]:
run_agent("What is 23 * 7?")

In [ ]:
run_agent("What is the square root of 144?")

In [ ]:
run_agent("Who discovered penicillin?")

In [ ]:
run_agent("When was the Eiffel Tower built?")

In [ ]:
# Your code here ...

# 🧩 Challenge: Add Your Own Tool (10 minutes)

Now it’s your turn!

Add a **new tool** that your agent can use.  
Some ideas:
- A `reverse(text)` tool that reverses words.
- A `coin_flip()` tool that randomly chooses between "Heads"/"Tails".
- A `word_count(text)` tool that counts words.

---

### 🧠 Steps

1. Define your function (like `def reverse(text): ...`).
2. Add it to the `tools` dictionary.
3. Try prompting the agent to use it!

## Example: Word Count Tool

In [ ]:
def reverse(text: str) -> str:
    """Reverses the order of words in the input text."""
    try:
        return " ".join(reversed(text.split()))
    except Exception as e:
        return f"Error: {e}"

# Don't forget to add your new tool to the dictionary
tools["reverse"] = reverse

print("✅ Added new tool: reverse()")

In [ ]:
run_agent("Reverse the following text: 'Artificial intelligence enables creative problem solving!'")

In [ ]:
# Your code here ...

# 🎭 Activity: Change the Agent's Personality (5-10 minutes)

In the system prompt of `run_agent()`, change the **tone** of the agent.  
You can make it:

- a **teacher** explaining steps in detail,  
- a **detective** solving mysteries,  
- a **medieval knight**, or  
- a **robotic assistant** giving short answers.

---

🧠 Try it out:
1. Create a variable for the new tone and assing it with `agent_tone=<your_new_tone>` when calling `run_agent`:
   > teacher_tone = "You are a curious teacher who explains your reasoning out loud before answering."
2. Experiment with different tones and see how your agent behaves!

## Example: Gen-Z Agent

In [ ]:
# Define a different personality tone than what is currently the default

gen_z_tone = "You are a typical Generation-Z young adult who explains your reasoning out loud (in internet/meme language) before answering."

In [ ]:
# Run the agent with example questions as querys and the gen-Z tone setting

print("___ Gen-Z Agent ___")
run_agent(query="What is the square root of 144?", agent_tone=gen_z_tone)

print("------")
run_agent(query="Who discovered penicillin?", agent_tone=gen_z_tone)

In [ ]:
# Your code here ...

# 🔍 Challenge: Multi-Tool Agent (Optional)

Can you make the agent **combine multiple tools**?

Example idea:
> "What is the population of France divided by the population of Germany?"

You could:
1. Use the `wiki` tool twice (once per country) to extract the population numbers (roughly),
2. Use the calculator to divide them.

Tip: You can allow more steps by increasing the `max_steps` argument in `run_agent()`.

In [ ]:
run_agent("What is the population of France divided by the population of Germany?")

In [ ]:
# Your code here ...

# 🏁 Wrap-Up

🎉 Congratulations — you’ve just built a working AI agent!

You’ve learned:
- How an agent reasons using the **Thought–Action–Observation** loop.  
- How to integrate **tools** like a calculator or Wikipedia search.  
- How to **extend** the system with your own creative ideas.  

---

💬 **Next steps**
- Try connecting to real APIs (e.g., [Open-Meteo](https://open-meteo.com/), [Yahoo Finance](https://github.com/ranaroussi/yfinance)).  
- Build a more advanced multi-tool agent with [memory](https://huggingface.co/docs/smolagents/en/tutorials/memory?utm_source=chatgpt.com).  
- Or use your gained insights to get into building custom AI assistants with [LibreChat](https://www.librechat.ai/docs/features/agents).

---

Thanks for participating — and have fun experimenting!